# TensorFlow.js

* https://www.tensorflow.org/javascript/
* https://js.tensorflow.org/

## Installation

In [1]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflowjs

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

## Prepare Dataset

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Build Model

In [0]:
sess = tf.InteractiveSession()

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

### Hypterparameter

In [0]:
n_input = 784
n_node = 256
n_out = 10

In [0]:
x = tf.placeholder(tf.float32, [None, n_input], name="x")

In [0]:
y_ = tf.placeholder(tf.float32, [None, n_out])

In [0]:
W = weight_variable([n_input, n_node])
b = bias_variable([n_node])
layer_h = tf.nn.relu(tf.matmul(x, W) + b)

In [0]:
W_out = bias_variable([n_node, n_out])
b_out =bias_variable([n_out])
y = tf.nn.relu(tf.matmul(layer_h, W_out) + b_out)

In [0]:
softmax = tf.nn.softmax(y, name="softmax")

## Loss Function

In [0]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

In [0]:
correct_prediction = tf.equal(tf.argmax(softmax, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Optimazation Algorithm

In [0]:
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

## Train

In [17]:
tf.global_variables_initializer().run()
for i in range(2000):
  batch = mnist.train.next_batch(50)
  if i % 200 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x: batch[0], y_: batch[1]})
    print('step %d, training accuracy %g' % (i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

print('test accuracy %g' % accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels}))

step 0, training accuracy 0.08
step 200, training accuracy 0.94
step 400, training accuracy 0.96
step 600, training accuracy 0.92
step 800, training accuracy 0.9
step 1000, training accuracy 0.98
step 1200, training accuracy 0.94
step 1400, training accuracy 0.98
step 1600, training accuracy 0.98
step 1800, training accuracy 0.98
test accuracy 0.9685


## Build SavedModel

In [18]:
!ls -l

total 8
drwxr-xr-x 1 root root 4096 Mar 13 21:48 datalab
drwxr-xr-x 2 root root 4096 Apr  6 09:28 MNIST_data


In [0]:
# !rm -rf saved_model/
# !rm -rf web_model/

In [20]:
tf.saved_model.simple_save(sess, "./saved_model", 
                           inputs={"x": x, }, outputs={"softmax": softmax, })

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'./saved_model/saved_model.pb'


In [21]:
!ls -l

total 12
drwxr-xr-x 1 root root 4096 Mar 13 21:48 datalab
drwxr-xr-x 2 root root 4096 Apr  6 09:28 MNIST_data
drwxr-xr-x 3 root root 4096 Apr  6 09:41 saved_model


## Converting a SavedModel to a web-friendly format

In [23]:
!tensorflowjs_converter --input_format=tf_saved_model \
  --output_node_names="softmax" \
  --saved_model_tags=serve ./saved_model \
  ./web_model

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Instructions for updating:
Use the retry module or similar alternatives.
2018-04-06 09:41:35.949558: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Converted 4 variables to const ops.
2018-04-06 09:41:36.174428: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:221] Return status of optimizer model_pruner: OK. Graph size before: 16 nodes, 15 edges. Graph size after: 12 nodes, 11 edges.
2018-04-06 09:41:36.174467: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:221] Return status of optimizer constant folding: OK. Graph size before: 12 nodes, 11 edges.

In [24]:
!ls -l

total 16
drwxr-xr-x 1 root root 4096 Mar 13 21:48 datalab
drwxr-xr-x 2 root root 4096 Apr  6 09:28 MNIST_data
drwxr-xr-x 3 root root 4096 Apr  6 09:41 saved_model
drwxr-xr-x 2 root root 4096 Apr  6 09:41 web_model


In [25]:
!ls -l web_model/

total 804
-rw-r--r-- 1 root root 814120 Apr  6 09:41 group1-shard1of1
-rw-r--r-- 1 root root    642 Apr  6 09:41 tensorflowjs_model.pb
-rw-r--r-- 1 root root    292 Apr  6 09:41 weights_manifest.json


## Download Model

In [0]:
from google.colab import files

In [27]:
!zip web_model -r web_model/

  adding: web_model/ (stored 0%)
  adding: web_model/weights_manifest.json (deflated 58%)
  adding: web_model/tensorflowjs_model.pb (deflated 53%)
  adding: web_model/group1-shard1of1 (deflated 7%)


In [28]:
!ls -l

total 756
drwxr-xr-x 1 root root   4096 Mar 13 21:48 datalab
drwxr-xr-x 2 root root   4096 Apr  6 09:28 MNIST_data
drwxr-xr-x 3 root root   4096 Apr  6 09:41 saved_model
drwxr-xr-x 2 root root   4096 Apr  6 09:41 web_model
-rw-r--r-- 1 root root 756693 Apr  6 09:42 web_model.zip


In [0]:
files.download("web_model.zip")

---